In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import albumentations as A
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
from tqdm.auto import tqdm

# Mount google drive
from google.colab import drive
drive.mount('gdrive')

## Rename images

We are going to make 20 new version of every single image by the means of random rotation and horizontal/vertical flipping. Therefore, we need to rename images so that the number of two neighboring ones differs by 21. After that, images with numbers from 1 to 21 will correspond to the first image (image with original number 1), images with numbers from 22 to 42 will correspond to the second image (image with original number 2) and so on.          

In [ ]:
# Folders with NIR images
nir_6_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ИК_больные/6 дней после заражения'
nir_26_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ИК_больные/26 дней после заражения'

# Folders with RGB images
rgb_6_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ВИД_больные/6 дней после заражения'
rgb_26_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ВИД_больные/26 дней после заражения'

# Folders with masks
mask_6_days_folder = '/content/gdrive/MyDrive/Cropped Potato/Разметка/6 дней после заражения/Разметка картинки'
mask_26_days_folder = '/content/gdrive/MyDrive/Cropped Potato/Разметка/26 дней после заражения/Разметка картинки'

In [ ]:
len(os.listdir(nir_6_days_folder)), len(os.listdir(nir_26_days_folder))

(64, 57)

In [ ]:
len(os.listdir(rgb_6_days_folder)), len(os.listdir(rgb_26_days_folder))

(64, 57)

In [ ]:
len(os.listdir(mask_6_days_folder)), len(os.listdir(mask_26_days_folder))

(64, 57)

In [ ]:
# Make it in reverse order to avoid problems with names collision

# NIR images, 6 days after infection
for i, image_name in enumerate(os.listdir(nir_6_days_folder)[::-1]):
    image_path = os.path.join(nir_6_days_folder, image_name)
    os.rename(image_path, f'{nir_6_days_folder}/IMG_{21 * (63 - i) + 1}.JPG')

In [ ]:
# RGB images, 6 days after infection
for i, image_name in enumerate(os.listdir(rgb_6_days_folder)[::-1]):
    image_path = os.path.join(rgb_6_days_folder, image_name)
    os.rename(image_path, f'{rgb_6_days_folder}/IMG_{21 * (63 - i) + 1}.JPG')

In [ ]:
# NIR images, 26 days after infection
for i, image_name in enumerate(os.listdir(nir_26_days_folder), start=64):
    image_path = os.path.join(nir_26_days_folder, image_name)
    os.rename(image_path, f'{nir_26_days_folder}/IMG_{21 * i + 1}.JPG')

In [ ]:
# RGB images, 26 days after infection
for i, image_name in enumerate(os.listdir(rgb_26_days_folder), start=64):
    image_path = os.path.join(rgb_26_days_folder, image_name)
    os.rename(image_path, f'{rgb_26_days_folder}/IMG_{21 * i + 1}.JPG')

In [ ]:
# Masks, 6 days after infection
for i, image_name in enumerate(os.listdir(mask_6_days_folder)[::-1]):
    image_path = os.path.join(mask_6_days_folder, image_name)
    os.rename(image_path, f'{mask_6_days_folder}/IMG_{21 * (63 - i) + 1}.png')

In [ ]:
# Masks, 26 days after infection
for i, image_name in enumerate(os.listdir(mask_26_days_folder), start=64):
    image_path = os.path.join(mask_26_days_folder, image_name)
    os.rename(image_path, f'{mask_26_days_folder}/IMG_{21 * i + 1}.png')

In [ ]:
# Check if we lost some images
len(os.listdir(nir_6_days_folder)), len(os.listdir(nir_26_days_folder))

(64, 57)

In [ ]:
len(os.listdir(rgb_6_days_folder)), len(os.listdir(rgb_26_days_folder))

(64, 57)

In [ ]:
len(os.listdir(mask_6_days_folder)), len(os.listdir(mask_26_days_folder))

(64, 57)

## Make augmentation

In [ ]:
transforms = A.Compose(
    [
        A.Rotate(limit=90, p=0.9, border_mode=cv2.BORDER_CONSTANT),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ]
    )

In [ ]:
nir_6_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ИК_больные/6 дней после заражения'
nir_26_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ИК_больные/26 дней после заражения'

rgb_6_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ВИД_больные/6 дней после заражения'
rgb_26_days_folder = '/content/gdrive/MyDrive/Cropped Potato/2023_ВИД_больные/26 дней после заражения'

mask_6_days_folder = '/content/gdrive/MyDrive/Cropped Potato/Разметка/6 дней после заражения/Разметка картинки'
mask_26_days_folder = '/content/gdrive/MyDrive/Cropped Potato/Разметка/26 дней после заражения/Разметка картинки'

In [ ]:
# perform augmentation transofmrs 20 times for every image and save it on google drive
for img_path in tqdm(os.listdir(nir_6_days_folder)):


    i = int(img_path.split('.')[0].split('_')[1]) # Number of image

    nir_img = np.array(Image.open(os.path.join(nir_6_days_folder, img_path))) # shape = [512, 512, 3]
    rgb_img = np.array(Image.open(os.path.join(rgb_6_days_folder, img_path)))

    img = np.concatenate([nir_img, rgb_img], axis=2)      # Concatenate along the last axis -> shape = [512, 512, 6]

    mask = np.array(Image.open(os.path.join(mask_6_days_folder, img_path.replace('JPG', 'png'))))

    for j in range(1, 21):
        augmented = transforms(image=img, mask=mask)

        augmented_img = augmented['image']
        augmented_nir_img = augmented_img[:, :, :3]
        augmented_rgb_img = augmented_img[:, :, 3:]

        augmented_mask = augmented['mask']

        Image.fromarray(augmented_nir_img).save(os.path.join(nir_6_days_folder, f'IMG_{i + j}.JPG'))
        Image.fromarray(augmented_rgb_img).save(os.path.join(rgb_6_days_folder, f'IMG_{i + j}.JPG'))
        Image.fromarray(augmented_mask).save(os.path.join(mask_6_days_folder, f'IMG_{i + j}.png'))


  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
len(os.listdir(mask_6_days_folder)), len(os.listdir(mask_26_days_folder))

(1344, 57)

In [ ]:
for img_path in tqdm(os.listdir(nir_26_days_folder)):

    i = int(img_path.split('.')[0].split('_')[1])

    nir_img = np.array(Image.open(os.path.join(nir_26_days_folder, img_path))) # shape = [512, 512, 3]
    rgb_img = np.array(Image.open(os.path.join(rgb_26_days_folder, img_path)))

    img = np.concatenate([nir_img, rgb_img], axis=2)      # Concatenate along the last axis -> shape = [512, 512, 6]

    mask = np.array(Image.open(os.path.join(mask_26_days_folder, img_path.replace('JPG', 'png'))))


    for j in range(1, 21):
        augmented = transforms(image=img, mask=mask)

        augmented_img = augmented['image']
        augmented_nir_img = augmented_img[:, :, :3]
        augmented_rgb_img = augmented_img[:, :, 3:]

        augmented_mask = augmented['mask']



        Image.fromarray(augmented_nir_img).save(os.path.join(nir_26_days_folder, f'IMG_{i + j}.JPG'))
        Image.fromarray(augmented_rgb_img).save(os.path.join(rgb_26_days_folder, f'IMG_{i + j}.JPG'))
        Image.fromarray(augmented_mask).save(os.path.join(mask_26_days_folder, f'IMG_{i + j}.png'))



  0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
len(os.listdir(mask_6_days_folder)), len(os.listdir(mask_26_days_folder))

(1344, 1197)